In [5]:
# version 0 works for the MIT experimental set up

import numpy as np
import cv2
#from common import clock,draw_str
import csv
import os
import time
import copy


MHI_DURATION = 1
MAX_TIME_DELTA = 0.5 # No usage for now
MIN_TIME_DELTA = 0.05 # No usage for now
THRESH_VALUE = 32
def reconstruction(ImMarker,ImReference):
    ImRec = copy.deepcopy(ImMarker)
    ImResult = copy.deepcopy(ImRec)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    
    width = np.size(ImMarker, 1) 
    height = np.size(ImMarker, 0)
    
    temp = np.zeros([height, width, 2],dtype = 'uint8')
    ImDilated = cv2.dilate(ImRec,kernel,iterations = 2)
    temp[:,:,0] = ImDilated
    temp[:,:,1] = ImReference
    ImRec = np.amin(temp,axis = 2)
    
    diff_16 = np.subtract(ImRec.astype(np.int16),ImResult.astype(np.int16))
    diff_16 = np.piecewise(diff_16,[diff_16<0,diff_16>=0],[lambda diff_16:0, lambda diff_16:diff_16])
    diff = diff_16.astype(np.uint8)
    #print(np.count_nonzero(diff))
    
    while np.count_nonzero(diff)>=3:
        ImDilated = cv2.dilate(ImRec,kernel,iterations = 1)
        temp[:,:,0] = ImDilated
        temp[:,:,1] = ImReference
        ImRec = np.amin(temp,axis = 2)
        diff_16 = np.subtract(ImRec.astype(np.int16),ImResult.astype(np.int16))
        diff_16 = np.piecewise(diff_16,[diff_16<0,diff_16>=0],[lambda diff_16:0, lambda diff_16:diff_16])
        diff = diff_16.astype(np.uint8)
        ImResult = ImRec
        #print(np.count_nonzero(abs(diff)))
        
    return ImResult
        
def largestComponent(fgmask):
    #find contours
    #_,contours,_ = cv2.findContours(fgmask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    contours,_ = cv2.findContours(fgmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    width = np.size(fgmask, 1) 
    height = np.size(fgmask, 0)
    
    temp = np.zeros([height, width, 3],dtype = 'uint8')
    
    if len(contours) > 0:

        areas = [] #list to hold all areas
        for contour in contours:
            ar = cv2.contourArea(contour)
            areas.append(ar)

        max_area = max(areas)
        max_area_index = areas.index(max_area) #index of the list element with largest area

        cnt = contours[max_area_index] #largest area contour
        cv2.drawContours(temp, [cnt], 0, (0, 255, 0), -1, maxLevel = 0)
        fgmask = np.squeeze(temp[:,:,1])
    return fgmask

# This function aims to extract features for various activities 
def video_feature_extraction_save(videoName, featureWriter, MIN_TIME_DELTA,MAX_TIME_DELTA,
                                  MHI_DURATION,THRESH_VALUE,DISPLAY=False): 
    cv2.namedWindow('rat activity recognition')
    visuals = ['input', 'frame_diff', 'motion_hist', 'grad_orient']
    # use MHI features (motion history intensity)
    visual_name = visuals[2]
  
    cam = cv2.VideoCapture(videoName)
    video_len = cam.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
    ret, frame = cam.read()
    
    
    h, w = frame.shape[:2]
    prev_frame = frame.copy()
    motion_history = np.zeros((h, w), np.float32)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[:,:,1] = 255
    ii = 0         
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    while (ii<video_len-5):
        ii += 1
        ret, frame = cam.read()
        
        frame_B = np.squeeze(frame[:,:,0])
        frame_G = np.squeeze(frame[:,:,1])
        frame_R = np.squeeze(frame[:,:,2])
    
        frame_diff = cv2.absdiff(frame, prev_frame)
        gray_diff = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
     
        ret, motion_mask = cv2.threshold(gray_diff, THRESH_VALUE, 1, cv2.THRESH_BINARY)
        timestamp = time.clock()
        cv2.updateMotionHistory(motion_mask, motion_history, timestamp, MHI_DURATION)
        mg_mask, mg_orient = cv2.calcMotionGradient(motion_history, MAX_TIME_DELTA, MIN_TIME_DELTA, apertureSize=5 )
        seg_mask, seg_bounds = cv2.segmentMotion(motion_history, timestamp, MAX_TIME_DELTA)

        if visual_name == 'input':
            vis = frame.copy()
        elif visual_name == 'frame_diff':
            vis = frame_diff.copy()
        elif visual_name == 'motion_hist':
            vis0 = np.uint8(np.clip((motion_history-(timestamp-MHI_DURATION)) / MHI_DURATION, 0, 1)*255)
            #junk,mei0 = cv2.threshold(vis0,1,255,cv2.THRESH_BINARY)

        elif visual_name == 'grad_orient':
            hsv[:,:,0] = mg_orient/2
            hsv[:,:,2] = mg_mask*255
            vis = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        
        ret,fgmask_R0 = cv2.threshold(frame_R,80,255,cv2.THRESH_BINARY_INV)
        ret,fgmask_G0 = cv2.threshold(frame_G,70,255,cv2.THRESH_BINARY_INV)
        ret,fgmask_B0 = cv2.threshold(frame_B,120,255,cv2.THRESH_BINARY_INV)
        
        fgmask_R0 = cv2.morphologyEx(fgmask_R0,cv2.MORPH_OPEN,kernel)
        fgmask_G0 = cv2.morphologyEx(fgmask_G0,cv2.MORPH_OPEN,kernel)
        fgmask_B0 = cv2.morphologyEx(fgmask_B0,cv2.MORPH_OPEN,kernel)
        
        fgmask_R = cv2.morphologyEx(fgmask_R0,cv2.MORPH_CLOSE,kernel)
        fgmask_G = cv2.morphologyEx(fgmask_G0,cv2.MORPH_CLOSE,kernel)
        fgmask_B = cv2.morphologyEx(fgmask_B0,cv2.MORPH_CLOSE,kernel)
        
        fgmask_bP = reconstruction(fgmask_R0,fgmask_G)#Blue paw
        fgmask_rP = fgmask_G-fgmask_bP # redPaw 
        
        fgmask_R[0:140,:] = 0
        fgmask_G[0:140,:] = 0
        fgmask_B[0:140,:] = 0
        
        fgmask_R[340:480,:] = 0
        fgmask_G[340:480,:] = 0
        fgmask_B[340:480,:] = 0
        
        fgmask_bP[0:140,:] = 0
        fgmask_rP[0:140,:] = 0
        
        fgmask_bP[340:480,:] = 0
        fgmask_rP[340:480,:] = 0

        fgmask_bP = largestComponent(fgmask_bP)
        fgmask_rP = largestComponent(fgmask_rP)
    
        #fgmask_red = fgmask_R*fgmask_B*fgmask_B
        ## Compute features
        M1 = cv2.moments(vis0)    
        Hu1 = cv2.HuMoments(M1)
        
        smallNum = [1e-200]*7 
        Hu1 = Hu1 + smallNum
        
        Hu1 = np.sign(Hu1)*np.log10(np.abs(Hu1))
        
            
        if M1['m00']!=0:
            cx2 = M1['m10']/M1['m00']
            cy2 = M1['m01']/M1['m00']
        else:
            cx2 = 0;
            cy2 = 0;     
                                       
  
        
        features = [Hu1[0][0],Hu1[1][0],Hu1[2][0],Hu1[3][0],Hu1[4][0],Hu1[5][0],Hu1[6][0]]
      
    
        featureWriter.writerow(features)
       
        prev_frame = frame.copy()
                            
        if DISPLAY:
            #draw_str(vis, (20, 20), visual_name)
       
            Bchan = cv2.cvtColor(frame_B,cv2.COLOR_GRAY2BGR)
            Gchan = cv2.cvtColor(frame_G,cv2.COLOR_GRAY2BGR)
            Rchan = cv2.cvtColor(frame_R,cv2.COLOR_GRAY2BGR)
            
            #roi_B = cv2.cvtColor(fgmask_B,cv2.COLOR_GRAY2BGR)
            #roi_G = cv2.cvtColor(fgmask_G,cv2.COLOR_GRAY2BGR)
            #roi_R = cv2.cvtColor(fgmask_R,cv2.COLOR_GRAY2BGR)
            
            #roi_B = np.zeros([h, w,3],dtype = 'uint8')
            #roi_B[:,:,0] = fgmask_R
            #roi_G = np.zeros([h, w,3],dtype = 'uint8')
            #roi_G[:,:,1]= fgmask_G
            #roi_R = np.zeros([h, w,3],dtype = 'uint8')
            #roi_R[:,:,2] = fgmask_B
            
            roi_RGB = np.zeros([h, w,3],dtype = 'uint8')
            roi_RGB[:,:,0] = fgmask_B
            roi_RGB[:,:,1] = fgmask_G
            roi_RGB[:,:,2] = fgmask_R
            
            roi = np.zeros([h, w,3],dtype = 'uint8')
            roi[:,:,0] = fgmask_bP # blue channel
            roi[:,:,2] = fgmask_rP # red channel
    
            cnt_B,_ = cv2.findContours(np.array(fgmask_bP),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cnt_R,_ = cv2.findContours(np.array(fgmask_rP),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            if len(cnt_B) > 0:
                cv2.drawContours(frame, cnt_B, 0, (255, 0, 0), 3, maxLevel = 0)
            if len(cnt_R) > 0:
                cv2.drawContours(frame, cnt_R, 0, (0, 0, 255), 3, maxLevel = 0)
            
            
            #frame_results = np.concatenate((frame,roi,roi_RGB),axis=1)
            #frame_results = frame_results[::2,::2,:]
            #frame_FG = np.concatenate((Gchan,roi_G),axis=1) 
            #frame_FB = np.concatenate((Bchan,roi_B),axis=1) 
            #frame_FG1 = np.concatenate((Gchan,roi_G1),axis=1) 
                
            cv2.imshow('Video',frame)
               
            if 0xff & cv2.waitKey(1) == 27:
                break
            
    cam.release()
    cv2.destroyAllWindows()

        
if __name__ == '__main__':
    import sys
    from os import listdir


    fullVideoName = 'C:\\VideoData\\Rat_paw_placement_cylinder_20170223\\P47_stitched\\p47_2017-02-23.avi'
    featureSaveName = 'C:\\VideoData\\Rat_paw_placement_cylinder_20170223\\P47_stitched\\p47_2017-02-23_features.csv'    

    fout = open(featureSaveName, 'wb')
    featureWriter = csv.writer(fout,quoting=csv.QUOTE_NONE)

        #mhi_video_extraction(fullVideoName, MIN_TIME_DELTA,MAX_TIME_DELTA,MHI_DURATION,THRESH_VALUE) 
    video_feature_extraction_save(fullVideoName, featureWriter, MIN_TIME_DELTA,MAX_TIME_DELTA,MHI_DURATION,
                                      THRESH_VALUE,True)

    fout.close()
    